In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

2.4.1+cu121
12.1
True


In [2]:
import os
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from collections import deque, namedtuple
import copy
from swig import swig
from tqdm import tqdm

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# Enable autograd anomaly detection
torch.autograd.set_detect_anomaly(True)

In [3]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [19]:
class Actor(nn.Module):
    def __init__(self, input_shape, action_size):
        super(Actor, self).__init__()
        self.conv1 = nn.Conv2d(input_shape[2], 16, kernel_size=4, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1)

        conv_out_size = self._get_conv_output(input_shape)

        self.fc1 = nn.Linear(conv_out_size, 256)
        self.fc2 = nn.Linear(256, action_size)

    def _get_conv_output(self, shape):
        input = torch.rand(1, *shape)
        input = input.permute(0, 3, 1, 2)
        output = self.conv3(self.conv2(self.conv1(input)))
        return int(np.prod(output.size()))

    def forward(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(0)
        if x.size(1) != 3:
            x = x.permute(0, 3, 1, 2)
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        action = torch.tanh(self.fc2(x))  # Constrain output between -1 and 1
        return action


class Critic(nn.Module):
    def __init__(self, input_shape, action_size):
        super(Critic, self).__init__()
        self.conv1 = nn.Conv2d(input_shape[2], 16, kernel_size=4, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1)

        conv_out_size = self._get_conv_output(input_shape)

        self.fc1 = nn.Linear(conv_out_size + action_size, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 1)

    def _get_conv_output(self, shape):
        input = torch.rand(1, *shape)
        input = input.permute(0, 3, 1, 2)
        output = self.conv3(self.conv2(self.conv1(input)))
        return int(np.prod(output.size()))

    def forward(self, state, action):
        if state.dim() == 3:
            state = state.unsqueeze(0)
        if state.size(1) != 3:
            state = state.permute(0, 3, 1, 2)
        
        x = F.relu(self.conv1(state))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        x = torch.cat([x, action], dim=1)  # Concatenate state and action
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        Q_value = self.fc3(x)
        return Q_value

In [20]:
import gymnasium as gym
env = gym.make("CarRacing-v3")
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]
action_space = env.action_space
print("State shape: ", state_shape)
print("state size: ", state_size)
print("action size: ", action_size)
print("action space:", action_space)

State shape:  (96, 96, 3)
state size:  96
action size:  3
action space: Box([-1.  0.  0.], 1.0, (3,), float32)


In [21]:
learning_rate = 1e-3
minibatch_size = 128
discount_factor = 0.99
replay_buffer_size = int(1e5)
tau = 1e-3 

In [22]:
class ReplayMemory:
    def __init__(self, capacity, state_shape, action_dim, device):
        self.capacity = capacity
        self.device = device
        self.memory = deque(maxlen=capacity)
        self.state_shape = state_shape
        self.action_dim = action_dim

    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        
        # Pre-allocate numpy arrays
        states = np.zeros((batch_size, *self.state_shape), dtype=np.float32)
        actions = np.zeros((batch_size, self.action_dim), dtype=np.float32)
        rewards = np.zeros((batch_size, 1), dtype=np.float32)
        next_states = np.zeros((batch_size, *self.state_shape), dtype=np.float32)
        dones = np.zeros((batch_size, 1), dtype=np.float32)

        # Fill the numpy arrays
        for i, (state, action, reward, next_state, done) in enumerate(batch):
            states[i] = state
            actions[i] = action
            rewards[i] = reward
            next_states[i] = next_state
            dones[i] = done

        # Convert numpy arrays to tensors
        return (
            torch.from_numpy(states).pin_memory().to(self.device,non_blocking=True),
            torch.from_numpy(actions).to(self.device),
            torch.from_numpy(rewards).to(self.device),
            torch.from_numpy(next_states).to(self.device),
            torch.from_numpy(dones).to(self.device)
        )

    def __len__(self):
        return len(self.memory)

In [23]:
class OUNoise:
    def __init__(self, size, mu=0., theta=0.15, sigma=0.2):
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.reset()

    def reset(self):
        self.state = copy.copy(self.mu)

    def sample(self):
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(len(x))
        self.state = x + dx
        return self.state.tolist()  # Return as a list

In [24]:
class Agent:
    def __init__(self, state_shape, action_size, actor_lr=1e-4, critic_lr=1e-3, replay_buffer_size=10000, minibatch_size=32):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.state_shape = state_shape
        self.action_size = action_size

        # Actor network
        self.actor = Actor(state_shape, action_size).to(self.device)
        self.target_actor = Actor(state_shape, action_size).to(self.device)
        self.target_actor.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=actor_lr)

        # Critic network
        self.critic = Critic(state_shape, action_size).to(self.device)
        self.target_critic = Critic(state_shape, action_size).to(self.device)
        self.target_critic.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=critic_lr)

        # Replay buffer
        self.memory = ReplayMemory(replay_buffer_size, state_shape, action_size, self.device)
        self.minibatch_size = minibatch_size

    def act(self, state, noise_scale=0.1):
        """Get action using the actor network and add noise for exploration."""
        self.actor.eval()
        with torch.no_grad():
            state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
            action = self.actor(state_tensor).cpu().data.numpy().squeeze()
        self.actor.train()

        # Add noise for exploration
        noise = noise_scale * np.random.randn(self.action_size)
        action = action + noise
        return np.clip(action, -1, 1)

    def step(self, state, action, reward, next_state, done):
        """Save experience to replay memory and learn if enough samples."""
        self.memory.push(state, action, reward, next_state, done)
        if len(self.memory) > self.minibatch_size:
            experiences = self.memory.sample(self.minibatch_size)
            self.learn(experiences, 0.99)

    def learn(self, experiences, gamma):
        """Update actor and critic networks from sampled experiences."""
        states, actions, rewards, next_states, dones = experiences

        # Critic update: Q-value estimation
        next_actions = self.target_actor(next_states)
        Q_targets_next = self.target_critic(next_states, next_actions).detach()
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        Q_expected = self.critic(states, actions)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Actor update: Maximize Q-value predicted by the critic
        actions_pred = self.actor(states)
        actor_loss = -self.critic(states, actions_pred).mean()
        
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Update target networks
        self.soft_update(self.critic, self.target_critic, 1e-3)
        self.soft_update(self.actor, self.target_actor, 1e-3)

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters."""
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [25]:
agent = Agent(state_shape, action_size)

In [14]:
def train_agent(agent, env, number_episodes=2000, max_timesteps_per_episode=1000, target_score=200.0):
    scores_window = deque(maxlen=100)
    
    noise_stddev = 0.2  # Standard deviation for noise
    noise_decay = 0.995  # Noise decay over episodes
    noise_min = 0.01  # Minimum noise level for exploration

    for episode in tqdm(range(1, number_episodes + 1), desc="Training Progress"):
        state, _ = env.reset()
        score = 0
        
        noise = np.random.normal(0, noise_stddev, agent.action_size)
        
        for t in range(max_timesteps_per_episode):
            action = agent.act(state)  # Get action from the policy (Actor network)
            action = action + noise  # Add exploration noise
            action = np.clip(action, -1, 1)  # Clip action to be within the valid range
            
            next_state, reward, done, _, _ = env.step(action)
            
            agent.step(state, action, reward, next_state, done)
            
            state = next_state
            score += reward
            
            if done:
                break
        
        scores_window.append(score)
        noise_stddev = max(noise_min, noise_stddev * noise_decay)  # Decay the noise
        
        if episode % 100 == 0:
            avg_score = np.mean(scores_window)
            print(f'\rEpisode {episode}\tAverage Score: {avg_score:.2f}\tNoise StdDev: {noise_stddev:.2f}')
            torch.save(agent.actor.state_dict(), f'checkpoint_actor.pth')
            torch.save(agent.critic.state_dict(), f'checkpoint_critic.pth')
        
        if np.mean(scores_window) >= target_score:
            print(f'\nEnvironment solved in {episode-100:d} episodes!\tAverage Score: {np.mean(scores_window):.2f}')
            torch.save(agent.actor.state_dict(), 'checkpoint_actor_final.pth')
            torch.save(agent.critic.state_dict(), 'checkpoint_critic_final.pth')
            return True
    
    print(f'\nEnvironment not solved. Final Average Score: {np.mean(scores_window):.2f}')
    torch.save(agent.actor.state_dict(), 'checkpoint_actor_final_us.pth')
    torch.save(agent.critic.state_dict(), 'checkpoint_critic_final_us.pth')
    return False

# Usage
train_agent(agent, env)

Training Progress:   0%|          | 2/2000 [01:36<26:54:44, 48.49s/it]